In [50]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import pprint

In [51]:
client = MongoClient('localhost', 27017)

In [52]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [53]:
db = client.payroll_data

In [54]:
payroll = pd.read_csv('merged_payroll_data.csv')
payroll.head()

,_id,First_Name,Last_Name,Dpt_ID,Manager,Tax_ID,department_Name,location,budget,base_salary,bonus,deductions,tax_rate,exemptions,witholdings
0,0,Jim,Reynolds,0,John,0,Sales,New York,500000,127708,4000,1000,24,600,300
1,1,Sarah,Brown,1,John,4,Marketing,Los Angeles,400000,55192,2000,1500,16,400,260
2,2,Ben,Smith,2,Miranda,5,Finance,Chicago,600000,100057,2000,1250,22,550,295
3,3,Kate,Dickson,0,Miranda,3,Sales,New York,500000,123349,4000,1300,12,250,245
4,4,Al,White,3,John,2,Human Resources,Houston,300000,71440,700,1000,10,200,230


In [55]:
payroll_dict = payroll.to_dict('records')
payroll_dict

[{'_id': 0,
  'First_Name': 'Jim',
  'Last_Name': 'Reynolds',
  'Dpt_ID': 0,
  'Manager': 'John',
  'Tax_ID': 0,
  'department_Name': 'Sales',
  'location': 'New York',
  'budget': 500000,
  'base_salary': 127708,
  'bonus': 4000,
  'deductions': 1000,
  'tax_rate': 24,
  'exemptions': 600,
  'witholdings': 300},
 {'_id': 1,
  'First_Name': 'Sarah',
  'Last_Name': 'Brown',
  'Dpt_ID': 1,
  'Manager': 'John',
  'Tax_ID': 4,
  'department_Name': 'Marketing',
  'location': 'Los Angeles',
  'budget': 400000,
  'base_salary': 55192,
  'bonus': 2000,
  'deductions': 1500,
  'tax_rate': 16,
  'exemptions': 400,
  'witholdings': 260},
 {'_id': 2,
  'First_Name': 'Ben',
  'Last_Name': 'Smith',
  'Dpt_ID': 2,
  'Manager': 'Miranda',
  'Tax_ID': 5,
  'department_Name': 'Finance',
  'location': 'Chicago',
  'budget': 600000,
  'base_salary': 100057,
  'bonus': 2000,
  'deductions': 1250,
  'tax_rate': 22,
  'exemptions': 550,
  'witholdings': 295},
 {'_id': 3,
  'First_Name': 'Kate',
  'Last_Name'

In [56]:
payroll_collection = db["payroll_collection"]

In [57]:
payroll_collection.insert_many(payroll_dict)

In [58]:
query = {'First_Name': 'Evelyn'}
payroll_collection.find_one(query)

{'_id': 19,
 'First_Name': 'Evelyn',
 'Last_Name': 'Moore',
 'Dpt_ID': 2,
 'Manager': 'Lucy',
 'Tax_ID': 3,
 'department_Name': 'Finance',
 'location': 'Chicago',
 'budget': 600000,
 'base_salary': 81959,
 'bonus': 950,
 'deductions': 1650,
 'tax_rate': 12,
 'exemptions': 250,
 'witholdings': 245}

In [59]:
new_dict = {'_id': 28,
 'First_Name': 'New',
 'Last_Name': 'Person',
 'Dpt_ID': 6,
 'Manager': 'Manager',
 'Tax_ID': 7,
 'department_Name': 'Dpt',
 'location': 'Chicago',
 'budget': 1600000,
 'base_salary': 181959,
 'bonus': 1950,
 'deductions': 11650,
 'tax_rate': 112,
 'exemptions': 1250,
 'witholdings': 1245}

In [60]:
payroll_collection.insert_one(new_dict)

## Nest tables within each other instead of merging

In [61]:
new_collection = db.nested_payroll

In [62]:
departments = pd.read_csv('departments_table.csv')
departments_dict = departments.to_dict('records')

employees = pd.read_csv('employees_table.csv')
employees_dict = employees.to_dict('records')

salaries = pd.read_csv('salaries_table.csv')
salaries_dict = salaries.to_dict('records')

taxes = pd.read_csv('taxes_table.csv')
taxes_dict = taxes.to_dict('records')

In [63]:
emp_and_sal = pd.read_csv('employee_and_salary.csv')
emp_and_sal_copy = emp_and_sal.copy()
emp_and_sal_copy = emp_and_sal_copy.drop('salary_id', axis=1)
emp_and_sal_copy.head()

,_id,First_Name,Last_Name,Dpt_ID,Manager,Tax_ID,base_salary,bonus,deductions
0,0,Jim,Reynolds,0,John,0,127708,4000,1000
1,1,Sarah,Brown,1,John,4,55192,2000,1500
2,2,Ben,Smith,2,Miranda,5,100057,2000,1250
3,3,Kate,Dickson,0,Miranda,3,123349,4000,1300
4,4,Al,White,3,John,2,71440,700,1000


In [64]:
emp_and_sal_copy_dict = emp_and_sal_copy.to_dict('records')

In [65]:
emp_sal_tax = pd.read_csv('emp_sal_taxes.csv')
emp_sal_tax_copy = emp_sal_tax.copy()
emp_sal_tax_copy = emp_sal_tax_copy.drop('salary_id', axis=1)
emp_sal_tax_copy.head()

,_id,First_Name,Last_Name,Dpt_ID,Manager,base_salary,bonus,deductions,Tax_ID,tax_rate,exemptions,witholdings
0,0,Jim,Reynolds,0,John,127708,4000,1000,0,24,600,300
1,1,Sarah,Brown,1,John,55192,2000,1500,4,16,400,260
2,2,Ben,Smith,2,Miranda,100057,2000,1250,5,22,550,295
3,3,Kate,Dickson,0,Miranda,123349,4000,1300,3,12,250,245
4,4,Al,White,3,John,71440,700,1000,2,10,200,230


In [66]:
emp_sal_tax_copy_dict = emp_sal_tax_copy.to_dict('records')

***** <b>`emp_sal_tax_nested_dict` is the dictionary that has the columns from the employee, salary, and taxes table. The department table is nested inside the `Dpt_ID` column <b>
    

In [67]:
#nest only department table into the combined employees, taxes, and salary data 
emp_sal_tax_nested_dict = emp_sal_tax_copy_dict.copy()
for employee in emp_sal_tax_nested_dict:
    dept_id = employee['Dpt_ID']
    employee['Dpt_ID'] = departments_dict[dept_id]
emp_sal_tax_nested_dict

[{'_id': 0,
  'First_Name': 'Jim',
  'Last_Name': 'Reynolds',
  'Dpt_ID': {'department_ID': 0,
   'department_Name': 'Sales',
   'location': 'New York',
   'budget': 500000},
  'Manager': 'John',
  'base_salary': 127708,
  'bonus': 4000,
  'deductions': 1000,
  'Tax_ID': 0,
  'tax_rate': 24,
  'exemptions': 600,
  'witholdings': 300},
 {'_id': 1,
  'First_Name': 'Sarah',
  'Last_Name': 'Brown',
  'Dpt_ID': {'department_ID': 1,
   'department_Name': 'Marketing',
   'location': 'Los Angeles',
   'budget': 400000},
  'Manager': 'John',
  'base_salary': 55192,
  'bonus': 2000,
  'deductions': 1500,
  'Tax_ID': 4,
  'tax_rate': 16,
  'exemptions': 400,
  'witholdings': 260},
 {'_id': 2,
  'First_Name': 'Ben',
  'Last_Name': 'Smith',
  'Dpt_ID': {'department_ID': 2,
   'department_Name': 'Finance',
   'location': 'Chicago',
   'budget': 600000},
  'Manager': 'Miranda',
  'base_salary': 100057,
  'bonus': 2000,
  'deductions': 1250,
  'Tax_ID': 5,
  'tax_rate': 22,
  'exemptions': 550,
  'wit

In [68]:
nested_dpt_only_collection = db.nested_department

In [69]:
#return salary of all employees 
pipeline = [
    {
        "$addFields": {
            "salary": {
                "$subtract": [
                    {
                        "$add": ["$base_salary", "$bonus"]
                    },
                    "$deductions"
                ]
            }
        }
    },
    {
        "$addFields": {
            "salary_after_tax": {
                "$multiply": [
                    "$salary",
                    {
                        "$subtract": [1, {"$divide": ["$tax_rate", 100]}]
                    }
                ]
            }
        }
    }
]


result = list(nested_dpt_only_collection.aggregate(pipeline))


#create df instead of print
for employee in result:
    print(f"Employee: {employee['First_Name']} {employee['Last_Name']}")
    print(f"Base Salary: ${employee['base_salary']}")
    print(f"Bonus: ${employee['bonus']}")
    print(f"Deductions: ${employee['deductions']}")
    print(f"Tax Rate: {employee['tax_rate']}%")
    print(f"Salary without Tax: ${employee['salary']}")
    print(f"Salary after Tax: ${employee['salary_after_tax']}")
    print("\n")

Employee: Jim Reynolds
Base Salary: $127708
Bonus: $4000
Deductions: $1000
Tax Rate: 24%
Salary without Tax: $130708
Salary after Tax: $99338.08


Employee: Sarah Brown
Base Salary: $55192
Bonus: $2000
Deductions: $1500
Tax Rate: 16%
Salary without Tax: $55692
Salary after Tax: $46781.28


Employee: Ben Smith
Base Salary: $100057
Bonus: $2000
Deductions: $1250
Tax Rate: 22%
Salary without Tax: $100807
Salary after Tax: $78629.46


Employee: Kate Dickson
Base Salary: $123349
Bonus: $4000
Deductions: $1300
Tax Rate: 12%
Salary without Tax: $126049
Salary after Tax: $110923.12


Employee: Al White
Base Salary: $71440
Bonus: $700
Deductions: $1000
Tax Rate: 10%
Salary without Tax: $71140
Salary after Tax: $64026.0


Employee: Olivia Johnson
Base Salary: $70609
Bonus: $1000
Deductions: $900
Tax Rate: 10%
Salary without Tax: $70709
Salary after Tax: $63638.1


Employee: Emma Williams
Base Salary: $99100
Bonus: $1500
Deductions: $1800
Tax Rate: 12%
Salary without Tax: $98800
Salary after Tax:

In [70]:
pipeline = [
    {
        "$addFields": {
            "salary": {
                "$subtract": [
                    {
                        "$add": ["$base_salary", "$bonus"]
                    },
                    "$deductions"
                ]
            }
        }
    },
    {
        "$addFields": {
            "salary_after_tax": {
                "$multiply": [
                    "$salary",
                    {
                        "$subtract": [1, {"$divide": ["$tax_rate", 100]}]
                    }
                ]
            }
        }
    }
]

result = list(nested_dpt_only_collection.aggregate(pipeline))


data = []
columns = ['Employee', 'Base Salary', 'Bonus', 'Deductions', 'Tax Rate', 'Salary without Tax', 'Salary after Tax']

for employee in result:
    employee_data = [
        f"{employee['First_Name']} {employee['Last_Name']}",
        employee['base_salary'],
        employee['bonus'],
        employee['deductions'],
        employee['tax_rate'],
        employee['salary'],
        employee['salary_after_tax']
    ]
    data.append(employee_data)

df = pd.DataFrame(data, columns=columns)
df


,Employee,Base Salary,Bonus,Deductions,Tax Rate,Salary without Tax,Salary after Tax
0,Jim Reynolds,127708,4000,1000,24,130708,99338.08
1,Sarah Brown,55192,2000,1500,16,55692,46781.28
2,Ben Smith,100057,2000,1250,22,100807,78629.46
3,Kate Dickson,123349,4000,1300,12,126049,110923.12
4,Al White,71440,700,1000,10,71140,64026.00
5,Olivia Johnson,70609,1000,900,10,70709,63638.10
6,Emma Williams,99100,1500,1800,12,98800,86944.00
7,Isabella Jones,57751,1500,1000,16,58251,48930.84
8,Liam Garcia,93757,2300,2200,22,93857,73208.46
9,Noah Miller,81228,1600,1700,32,81128,55167.04


In [71]:
#workflow diagram google slides

#calculate salary based on _id
def calculate_salary(employee_id):
    pipeline = [
        {
            "$match": {
                "_id": employee_id
            }
        },
        {
            "$addFields": {
                "salary": {
                    "$subtract": [
                        {
                            "$add": ["$base_salary", "$bonus"]
                        },
                        "$deductions"
                    ]
                }
            }
        },
        {
            "$addFields": {
                "salary_after_tax": {
                    "$multiply": [
                        "$salary",
                        {
                            "$subtract": [1, {"$divide": ["$tax_rate", 100]}]
                        }
                    ]
                }
            }
        }
    ]

    result = list(nested_dpt_only_collection.aggregate(pipeline))

    if len(result) > 0:
        employee = result[0]
        print(f"Employee ID #: {employee['_id']}")
        print(f"Employee Name: {employee['First_Name']} {employee['Last_Name']}")
        print(f"Base Salary: ${employee['base_salary']}")
        print(f"Bonus: ${employee['bonus']}")
        print(f"Deductions: ${employee['deductions']}")
        print(f"Tax Rate: {employee['tax_rate']}%")
        print(f"Salary: ${employee['salary']}")
        print(f"Salary after Tax: ${employee['salary_after_tax']}")
    else:
        print("Employee not found.")
calculate_salary(5)

Employee ID #: 5
Employee Name: Olivia Johnson
Base Salary: $70609
Bonus: $1000
Deductions: $900
Tax Rate: 10%
Salary: $70709
Salary after Tax: $63638.1


In [73]:
payroll_collection.delete_many({})
new_collection.delete_many({})